<a href="https://colab.research.google.com/github/lucacbbi/Projetos_data_science/blob/main/Webscraping_forecast_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests_html

In [10]:
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession

import re
import regex as re
import calendar

import time as time_sleep
from datetime import date,datetime,timedelta
import pandas as pd
import numpy as np

class Wheather_forecast:

    def clean_str_letters(self,x):

        """
            Clean string of the letters. 
        """
        x = re.sub('[^0-9]', '', x)
        return x


    def get_request(self):

        # CONNECTION
        """
            Treats the any request exception. 
            If has any exception the command 'exception' will execute 
            procedure of print "Connection refused by the server ..." 
            and the command 'continue' makes the loop execute request 
            again wicht the loop.
        """
        while True:            
            try:
                session = HTMLSession() # Initialize the GET requests
                url = 'https://weather.com/pt-BR/weather/hourbyhour/l/114d4a2a1e00929e75ae0f9b034e93bd346c1104b06a59ab9237119bcca0a301'
                r = session.get(url) # GET requests
                break
            except:# Error of connection
                print("Connection refused by the server...")
                time_sleep.sleep(5)
                continue

        return r


    def build_data_scraping(self):

        # Data frame of wheater
        df_wheather = pd.DataFrame(columns=['DATE','Temperatura(C°)','Humidade relativa do ar','Velocidade do vento(m/s)'])
        
        r = self.get_request()
        # Scraping 
        wind_humi_forecast = r.html.find('[data-testid="DetailsTable"]') # wind humi 6 9
        hour_temp_forecast = r.html.find('[data-testid="DetailsSummary"]') # hour temp 0 1

        for i,data in enumerate(zip(wind_humi_forecast,hour_temp_forecast),1):
            
            # Split vector containing datas
            wh = data[0].text.split() 
            ht = data[1].text.split() 

            temp = float(self.clean_str_letters(ht[1])) # Temperature
            humi = float(self.clean_str_letters(wh[9])) # Humidity
            wind = float(self.clean_str_letters(wh[6]))/3.6 # Wind

            # Date time of data
            date = datetime.now() + timedelta(hours=i)
            date_time = datetime(date.year,date.month,date.day,date.hour)
            # fit register 
            data = {'DATE':date_time,'Temperatura(C°)':temp,'Humidade relativa do ar':humi,'Velocidade do vento(m/s)':wind}            
            # Append data
            df_wheather = df_wheather.append(data,ignore_index=True)
            
        return df_wheather


    def request(self,hours):
        
        """ 
            Web scraping 

            >> Does scraping data of the website 'https://weather.com/pt-BR/weather/
            hourbyhour/l/114d4a2a1e00929e75ae0f9b034e93bd346c1104b06a59ab9237119bcca0a301', 
            format and build an time series weather forecast data frame.
        """
        while True: # Need for data prediction from the current time +1
            
            df_wheather = self.build_data_scraping()

            # >>> Buid and format Date Time 
            forecast_wheather = df_wheather.iloc[:hours].set_index('DATE')

            # >>> Verification
            """
                Verifies if data is updated comparing the first item index 
                of data frame to the current hour. If its different the loop 
                it will be break else it will be make a new requisition again 
                in loop.
            """

            today_now = datetime.now().hour
            nex_hour_forecast = forecast_wheather.iloc[[0]].index.hour[0]

            if(nex_hour_forecast == today_now):
                continue
            else:
                break

        return forecast_wheather



In [11]:
weather = Wheather_forecast()
weather.request(23)

,Temperatura(C°),Humidade relativa do ar,Velocidade do vento(m/s)
DATE,,,
2021-07-30 21:00:00,25.0,32.0,4.444444
2021-07-30 22:00:00,23.0,37.0,4.166667
2021-07-30 23:00:00,21.0,41.0,3.611111
2021-07-31 00:00:00,19.0,44.0,1.666667
2021-07-31 01:00:00,18.0,48.0,1.666667
2021-07-31 02:00:00,17.0,51.0,1.388889
2021-07-31 03:00:00,16.0,55.0,1.111111
2021-07-31 04:00:00,15.0,60.0,1.111111
2021-07-31 05:00:00,14.0,64.0,1.111111
